In [1]:
#importing system libraries
import os 
import sys
import warnings

#importing libraries for model making
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

In [2]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [18]:
if __name__ == "__main__":
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    csv_url = ("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv")
    
    try:
        data = pd.read_csv(csv_url, sep=";")
    
    except Exception as e:
        logger.exception("Unable to download training & test CAV, check your internet connection. Error: %s", e)
    
    train, test = train_test_split(data)
    
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = 0.5
    l1_ratio = 0.5
    
    with mlflow.start_run():
        
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        y_pred = lr.predict(test_x)
        
        (rmse, mae, r2) = eval_metrics(test_y, y_pred)
        
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        if tracking_url_type_store != "file":
            mlflow.sklearn.log_model(lr, "model", registered_model_name="Elasticnetwinemodel")
        else:
            mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.793164022927685
  MAE: 0.6271946374319586
  R2: 0.10862644997792636


In [19]:
!mlflow ui --backend-store-uri file:///Users/Swapnil/Documents/LocalPython/MLFLowDemo/mlrun

2023/07/21 00:49:33 ERROR mlflow.cli: Error initializing backend store
2023/07/21 00:49:33 ERROR mlflow.cli: [WinError 5] Access is denied: '\\Users\\Swapnil'
Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\envs\venv\lib\site-packages\mlflow\cli.py", line 426, in server
    initialize_backend_stores(backend_store_uri, registry_store_uri, default_artifact_root)
  File "C:\Users\HP\anaconda3\envs\venv\lib\site-packages\mlflow\server\handlers.py", line 279, in initialize_backend_stores
    _get_tracking_store(backend_store_uri, default_artifact_root)
  File "C:\Users\HP\anaconda3\envs\venv\lib\site-packages\mlflow\server\handlers.py", line 258, in _get_tracking_store
    _tracking_store = _tracking_store_registry.get_store(store_uri, artifact_root)
  File "C:\Users\HP\anaconda3\envs\venv\lib\site-packages\mlflow\tracking\_tracking_service\registry.py", line 39, in get_store
    return self._get_store_with_resolved_uri(resolved_store_uri, artifact_uri)
  File "C:\Users\HP\